# Pseudo-LiDAR Demo

## Setup

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from pseudolidar.pcl_generator import PointCloudGenerator
from pseudolidar.lidar_generator import PseudoLidarGenerator

In [ ]:
IMG_DIMS = (300, 400)
FOV = 100

camera_poses = {
    "cam_front": {
        "x": 1.3,
        "y": 0.0,
        "z": 2.3,
        "roll": 0.0,
        "pitch": 0.0,
        "yaw": 0.0
    },
    "cam_rear": {
        "x": -1.3,
        "y": 0.0,
        "z": 2.3,
        "roll": 0.0,
        "pitch": 0.0,
        "yaw": 180.0
    },
    "cam_left": {
        "x": 1.3,
        "y": 0.0,
        "z": 2.3,
        "roll": 0.0,
        "pitch": 0.0,
        "yaw": -60.0
    },
    "cam_right": {
        "x": 1.3,
        "y": 0.0,
        "z": 2.3,
        "roll": 0.0,
        "pitch": 0.0,
        "yaw": 60.0
    }
}

In [ ]:
def load_pose(extrinsics):
    sinval = np.sin(extrinsics['yaw'] * np.pi / 180.)
    cosval = np.cos(extrinsics['yaw'] * np.pi / 180.)
    Rz = np.array([[cosval, -sinval, 0],
                   [sinval, cosval, 0],
                   [0, 0, 1]])
    t = np.array([extrinsics['x'], extrinsics['y'], extrinsics['z']])
    
    T = np.eye(4)
    T[:3, :3] = Rz
    T[:3, 3] = Rz @ t
    return T

In [ ]:
# Load camera intrinsics
front_pose = load_pose(camera_poses['cam_front'])
left_pose = load_pose(camera_poses['cam_left'])
right_pose = load_pose(camera_poses['cam_right'])

In [ ]:
# Set up pseudo-LiDAR generator
lidar_generator = PseudoLidarGenerator([front_pose, left_pose, right_pose])
pcl_generator = PointCloudGenerator(fov=FOV, image_dims=IMG_DIMS)

## Simple Pseudo-Lidar

The following cells create and depict pseudo-LiDAR as proposed by Wang et al.

In [ ]:
# Create point clouds from the depth maps and fuse them in the vehicle's coordinate frame
pcls = []
pcls.append(pcl_generator.generate(os.path.join('demo_assets', 'depth_front.png'), max_depth=0.05, sparsity=3))
pcls.append(pcl_generator.generate(os.path.join('demo_assets', 'depth_left.png'), max_depth=0.03, sparsity=3))
pcls.append(pcl_generator.generate(os.path.join('demo_assets', 'depth_right.png'), max_depth=0.03, sparsity=3))
pcl = lidar_generator.generate(pcls)

In [ ]:
# Project into bird-eye-view and visualize
bev = PseudoLidarGenerator.post_process(pcl)
viz = PseudoLidarGenerator.to_grayscale_image(bev)

plt.imshow(viz, cmap='gray')

## Raycasting-based Pseudo-LiDAR

The following blocks generate and visualize our improved pseudo-LiDAR, which reconstructs a triangle mesh from the back-projected point clouds and raycasts it to simulate a LiDAR sensor.

In [ ]:
# Create point clouds from the depth maps and fuse them in the vehicle's coordinate frame
pcls = []
pcls.append(pcl_generator.generate(os.path.join('demo_assets', 'depth_front.png'), max_depth=0.05))
pcls.append(pcl_generator.generate(os.path.join('demo_assets', 'depth_left.png'), max_depth=0.03))
pcls.append(pcl_generator.generate(os.path.join('demo_assets', 'depth_right.png'), max_depth=0.03))
pcl = lidar_generator.generate(pcls)

In [ ]:
DOWNSAMPLE_VOXEL_SIZE = 0.5
NUM_LIDAR_CHANNELS = 32
NUM_MEASUREMENTS_PER_ROTATION = 250

# Reconstruct a triangle mesh and raycast it
mesh = PseudoLidarGenerator.reconstruct_mesh(pcl, voxel_size=DOWNSAMPLE_VOXEL_SIZE)
points = PseudoLidarGenerator.raycast_mesh(mesh, channels=NUM_LIDAR_CHANNELS, n_measurements=NUM_MEASUREMENTS_PER_ROTATION)

In [ ]:
# Project into bird-eye-view and visualize
bev = PseudoLidarGenerator.post_process(points)
viz = PseudoLidarGenerator.to_grayscale_image(bev)

plt.imshow(viz, cmap='gray')

## LiDAR Reference

The following visualizes the same scene captured by a CARLA LiDAR sensor

In [ ]:
# LiDAR visualization
lidar = np.load('demo_assets/lidar.npy')
lidar = lidar[:, [1, 0, 2]]
lidar_bev = PseudoLidarGenerator.post_process(lidar)
lidar_viz = PseudoLidarGenerator.to_grayscale_image(lidar_bev)

plt.imshow(lidar_viz, cmap='gray')
